# Part 1 : Data Preparation, Process, and Store Features

In [ ]:
# Uncomment and install s3fs, this is required to read CSV files from S3 directly into Pandas dataframe
# Once installed, please restart the Notebook Kernel (Kernel > Restart Kernel) before proceeding

#%pip install s3fs

<a id='all-up-overview'></a>

## Overview
* **[Notebook 1: Data Preparation, Process, and Store Features](./1-data-analysis-prep.ipynb)**
  * **[Getting started](#aud-getting-started)**
  * **[Analyzing Data and use SageMaker Data Wranger](#wrangler)**
  * **[SageMaker Feature Store](#aud-feature-store)**
  * **[Create Training DataSet](#dataset)**
  * **[Conclusion](#conclusion)**
* [Notebook 2: Amazon Fraud Detector Model Setup](./2-afd-model-setup.ipynb)
* [Notebook 3: Model training, deployment, real-time and batch inference](./3-afd-model-train-deploy.ipynb)
* [Notebook 4: Create an end-to-end pipeline](./4-afd-pipeline.ipynb)

The purpose of this notebook is to perform the Data Prep phase of the ML life cycle. The main Data Wrangling, data ingestion, and multiple transformations will be done through the SageMaker Studio Data Wrangler GUI.

In this notebook, we will take the `.flow` files that define the transformations to the raw data. and apply them using a SageMaker Processing job that will apply those transformations to the raw data deposited in the S3 bucket as `.csv` files.

<img src="images/nb1.png" width="600" height="800"/>

### Install required and/or update third-party libraries

In [ ]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler==2.4.0 imbalanced-learn==0.7.0 sagemaker==2.23.1 boto3==1.16.48

### Loading stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything printed then it's probably the first time you are running the notebook! 

In [ ]:
# %store -z  #deletes all previous stored variables
%store -r
%store

### Import libraries

In [ ]:
import json
import time
import boto3
import string
import sagemaker
import pandas as pd
import awswrangler as wr

from sagemaker.feature_store.feature_group import FeatureGroup

## 1. Getting started: Creating Resources <a id='aud-getting-started'></a>

[overview](#all-up-overview)
___
In order to successfully run this notebook you will need to create some AWS resources. 
First, an S3 bucket will be created to store all the data for this tutorial. 
Once created, you will then need to create an AWS Glue role using the IAM console then attach a policy to the S3 bucket to allow FeatureStore access to this notebook. If you've already run this notebook and are picking up where you left off, then running the cells below should pick up the resources you already created without creating any additional resources.

#### Add FeatureStore policy to Studio's execution role

<img src="images/iam-policies.png" width="500" height="600"/>


1. In a separate brower tab go to the IAM section of the AWS Console
2. Navigate to the Roles section and select the execution role you're using for your SageMaker Studio user
    * If you're not sure what role you're using, run the cell below to print it out
3. Attach the <font color='green'> AmazonSageMakerFeatureStoreAccess </font> policy to this role. Once attached, the changes take  effect immediately.

In [ ]:
print('SageMaker Role:', sagemaker.get_execution_role().split('/')[-1])
print('SageMaker Role:', sagemaker.get_execution_role())

### 1.1 Set region, boto3 and SageMaker SDK variables

In [25]:
#You can change this to a region of your choice
import sagemaker
region = sagemaker.Session().boto_region_name
%store region
print("Using AWS Region: {}".format(region))

Stored 'region' (str)
Using AWS Region: us-east-2


In [ ]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)

sagemaker_boto_client = boto_session.client('sagemaker')

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)

In [ ]:
"""
Note: if you are not running this notebook from SageMaker Studio or SageMaker Classic Notebooks you will need to instanatiate 
the sagemaker_execution_role_name with an AWS role that has SageMakerFullAccess and SageMakerFeatureStoreFullAccess
"""
sagemaker_execution_role_name = 'AmazonSageMaker-ExecutionRole-20210107T234882'
try:
    sagemaker_role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    sagemaker_role = iam.get_role(RoleName=sagemaker_execution_role_name)['Role']['Arn']
    print(f"\n instantiating sagemaker_role with supplied role name : {sagemaker_role}")

account_id = boto3.client('sts').get_caller_identity()["Account"]

### 1.2 Create a directory (prefix) in the SageMaker default S3 bucket

In [ ]:
from datetime import datetime
current_time = datetime.now()

if 'afd_bucket' not in locals():
    afd_bucket = sagemaker_session.default_bucket()
    afd_prefix = "amazon-fraud-detector"
    %store afd_bucket
    %store afd_prefix
    print(f'Using default bucket: {afd_bucket}... Initialized folder {afd_prefix}')
else:
    print(f'Bucket name already in local cache')

[**Optoinal**] If you want to use your own S3 bucket that's already existing, uncomment and utilize the following example code.

In [ ]:
'''
try:
    s3_client.create_bucket(Bucket=bucket, ACL='private', CreateBucketConfiguration={'LocationConstraint': region})
    print('Create S3 bucket: SUCCESS')
    
except Exception as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f'Using existing bucket: {bucket}/{prefix}')
    else:
        raise(e)
'''

In the following cell, we will initialize a few variables which will be used in either this Notebook or subsequent notebooks. 

In [ ]:
traing_job_output_path = f's3://{afd_bucket}/{afd_prefix}/training_jobs'
bias_report_1_output_path = f's3://{afd_bucket}/{afd_prefix}/clarify-bias-1'
bias_report_2_output_path = f's3://{afd_bucket}/{afd_prefix}/clarify-bias-2'
explainability_output_path = f's3://{afd_bucket}/{afd_prefix}/clarify-explainability'

train_data_uri = f's3://{afd_bucket}/{afd_prefix}/data/train/train.csv'
test_data_uri = f's3://{afd_bucket}/{afd_prefix}/data/test/test.csv'

## 2. Analyzing Data and use SageMaker Data Wranger <a id="wrangler"></a>
---

We will use two data sets in file `signup_attempts.csv` and `signup_outcomes.csv`. The `signup_attempts.csv` contains data about each signup session such as the user's ip_address, email address, city, state, address, and a timestamp of when the user attempted to sign-up for our service.

**<font color='red'>NOTE</font>: We are assuming that this dataset is historical from your environment. To train the model, Amazon Fraud Detector requires at least 10,000 records, with at least 500 of those records identified as fraudulent; however, the more records and variety of fraudulent examples you provide, the better. Refer to [this blog](https://aws.amazon.com/blogs/machine-learning/catching-fraud-faster-by-building-a-proof-of-concept-in-amazon-fraud-detector/) for more.**

The next data set i.e. `signup_outcomes.csv`, is the corresponding predictions of each of the signup attempts in the `signup_attempts.csv` file. In this case, we are assuming that this dataset is generated by your existing FD/FP service.

Finally, we will need to use a combination of these two data sets to construct our training data set required for Amazon Fraud Detector model training.

In [ ]:
# Importing required libraries.
import pandas as pd
import numpy as np

# load the signup_attempts dataset into a dataframe
df_signups = pd.read_csv("./data/signup_attempts.csv", index_col=0)
df_signups.head()

In [ ]:
# Importing required libraries.
import pandas as pd
import numpy as np

# load the signup_outcomes dataset into a dataframe
df_signups_pred = pd.read_csv("./data/signup_outcomes.csv", index_col=0)
# df_signups_pred.head()
df_signups_pred.isnull().sum()

As we can see that the `df_signups` dataset consists of a number of features including the `ip_address`, `email_address` and the `timestamp`. Note that the `timestamp` here is in Epoch/UNIX timestamp format. In order to be able to make it compatible for Amazon Fraud Detector model training, we need to convert it into either one of date formats listed below-

* %yyyy-%mm-%ddT%hh:%mm:%ssZ (ISO 8601 standard in UTC only with no milliseconds)
* %yyyy/%mm/%dd %hh:%mm:%ss (AM/PM)
* %mm/%dd/%yyyy %hh:%mm:%ss
* %mm/%dd/%yy %hh:%mm:%ss

For more information on the "Timestamp formats" check out the [documentation](https://docs.aws.amazon.com/frauddetector/latest/ug/online-fraud-insights.html)

We will use Amazon Sagemaker Data Wrangler to perform this transformation to convert the timestamp to a compatible date timestamp, in this case we will use `%yyyy/%mm/%dd %hh24:%mm:%ss` format. 

In [ ]:
df_signups.isnull().sum()

### 2.1 Upload raw data to S3
Before you can preprocess the raw data with Data Wrangler, it must exist in S3.

In [ ]:
s3_client.upload_file(Filename='data/signup_attempts.csv', Bucket=afd_bucket, Key=f'{afd_prefix}/data/raw/signup_attempts.csv')
s3_client.upload_file(Filename='data/signup_outcomes.csv', Bucket=afd_bucket, Key=f'{afd_prefix}/data/raw/signup_outcomes.csv')

### 2.2 Use Data Wrangler `flow` files to do transformations

As we can see that there are quite a few empty/null values for our dataset features for both the files. We will use SageMaker Data Wrangler to fill these missing values. We are going to perform these transformations on our `signup_attempts.csv` and `signup_outcomes.csv` file-

* First we will convert the `timestamp` feature from Epoch/Unix timestamp to a Amazon Fraud Detector compatible date time format in the `signup_attempts.csv` file. We will use a small piece of custom Pandas code for this transformation using "Custom Transform" in Data Wrangler.

```python
import pandas as pd
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.strftime('%m/%d/%y %H:%M:%S')
```

<img src="images/custom_transform.png" width="400"/>

**<font color='red'>NOTE</font>: Your entire dataset will be available in a built in variable named `df` within the Data Wrangler `.flow` file.**

* Rename the column `timestamp` to `EVENT_TIMESTAMP`. This is a requirement for Amazon Fraud Detector. For more information on this please refer to the [documentation](https://docs.aws.amazon.com/frauddetector/latest/ug/online-fraud-insights.html)

<img src="images/rename_column.png" width="400"/>

* We will fill missing values for the features. A sample missing value transformation looks like this -

<img src="images/handle_missing.png" width="400"/>

### 2.3 Update attributes within the  `.flow` file 
DataWrangler will generate a .flow file, in our case the file is named `signup_attempts.flow` and `signup_outcomes.flow` and they are included in this project. Each flow file contains a reference to an S3 bucket used during the Wrangling. This may be different from the one you have as a default in this notebook eg. if the Wrangling was done by someone else, you will probably not have access to their bucket and you now need to point to your own S3 bucket so you can actually load the `.flow` file into Wrangler or access the data.

After running the cell below you can open the `signup_attempts.flow` and `signup_outcomes.flow` file and export the data to S3 (described in the next step) or you can continue this guide using the provided `data/signup_attempts_preprocessed.csv` and `data/signup_outcomes_preprocessed.csv` files.

_Optionally, you may create your own Data Wrangler `.flow` file. Refer [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-getting-started.html) for step-by-step instructions on how to create one_.

In [ ]:
# Run this in case you want to run the .flow files to generate the preprocessed data sets
# if you want to save some time you may continue with this notebook by utilizing the preprocessed files already included in the data/ directory

signup_attempts_flow_template_file = "signup_attempts_flow_template"

with open(signup_attempts_flow_template_file, 'r') as f:
    variables   = {'bucket': afd_bucket, 'prefix': afd_prefix}
    template    = string.Template(f.read())
    signup_attempts_flow = template.substitute(variables)
    signup_attempts_flow = json.loads(signup_attempts_flow)

with open('signup_attempts.flow', 'w') as f:
    json.dump(signup_flow, f)
    
    
signup_outcomes_flow_template_file = "signup_outcomes_flow_template"

with open(signup_outcomes_flow_template_file, 'r') as f:
    variables   = {'bucket': afd_bucket, 'prefix': afd_prefix}
    template    = string.Template(f.read())
    signup_outcomes_flow = template.substitute(variables)
    signup_outcomes_flow = json.loads(signup_outcomes_flow)

with open('signup_outcomes.flow', 'w') as f:
    json.dump(signup_attempts_flow, f)

Our `.flow` file looks as shown below. Note that `.flow` visualization created with Data Wrangler is native to SageMaker Studio. You may still use your flow file in SageMaker Instances, however you may have to make updates to it using the method showed in the above code cell since the `.flow` file is based on underlying JSON data (Edit - `.flow` file in a text editor)

<img src="images/flow_file.png" width="400"/>

We will now run the flow file by first exporting it. There are a few options to export your flow file. In our case, we will export the results of the transformation defined in the flow file into our S3 bucket which will generate the pre-processed data set and save it in the bucket using a _SageMaker Processing Job_. If you select "Save to S3" SageMaker Studio will generate a New Notebook which you can tweak and run to generate the pre-processed data, you can also generate a Python code using the "Python Code". 

<img src="images/export-flow.png" width="600"/>

You can generate the notebook using the details mentioned above to generate the pre-processing export to S3 notebook or, continue using this tutorial with pre-processed data file that is already generated for you in the `/data` directory -- file name `signup_attempts_preprocessed.csv` and `signup_outcomes_preprocessed.csv`. To continue to use this file we will create a preprocessed directory in our S3 bucket and upload it there, ideally your Data Wrangler job would output the final processed file into this location. 

Once the files are in the designated S3 buckets we will load them into dataframes using Data Wrangler.

## 3. DataSets and Feature Types <a id='aud-datasets'></a>
[overview](#all-up-overview)
___

In [ ]:
signups_dtypes = {'ip_address': str,
 'email_address': str,
 'user_agent': str,
 'customer_city': str,
 'customer_state': str,
 'customer_postal': str,
 'customer_name': str,
 'customer_address': str,
 'phone_number': str,
 'EVENT_TIMESTAMP': str
 }

outcomes_dtypes = {'ip_address': str,
 'email_address': str,
 'EVENT_LABEL': str
}

In [ ]:
#======> This is your DataFlow output path if you decide to redo the work in DataFlow on your own
flow_output_path = 'YOUR_PRE_PROCESSED_DATA_S3_PATH_HERE'

try:
    # this will try to load the exported dataframes from the .flow files
    signups_s3_preprocessed_path = f'{flow_output_path}/signups'
    
    signups_preprocessed = wr.s3.read_csv(
        path=signups_s3_preprocessed_path, 
        dataset=True, 
        index_col=0,
        dtype=signups_dtypes)
    
    outcomes_s3_preprocessed_path = f'{flow_output_path}/outcomes'
    
    outcomes_preprocessed = wr.s3.read_csv(
        path=outcomes_s3_preprocessed_path, 
        dataset=True, 
        index_col=0,
        dtype=outcomes_dtypes)

except:
    # if the Data Wrangler job was not run, the claims and customers dataframes will be loaded from local copies
    timestamp = pd.to_datetime('now').timestamp()
    print('Unable to load Data Wrangler output. Loading local pre-processed files into s3 and dataframes...')
    
    signup_attempts_s3_path = f'{afd_prefix}/data/preprocessed/signups/signup_attempts_preprocessed.csv'
    signup_outcomes_s3_path = f'{afd_prefix}/data/preprocessed/outcomes/signup_outcomes_preprocessed.csv'
    
    # Upload the local pre-processed files to S3
    s3_client.upload_file(Filename='data/signup_outcomes_preprocessed.csv', Bucket=afd_bucket, Key=signup_outcomes_s3_path)
    s3_client.upload_file(Filename='data/signup_attempts_preprocessed.csv', Bucket=afd_bucket, Key=signup_attempts_s3_path)
    
    signups_preprocessed = wr.s3.read_csv(
        path=f's3://{afd_bucket}/{afd_prefix}/data/preprocessed/signups' , 
        dataset=True, 
#         index_col=0,
        dtype=signups_dtypes)
    
    # a timestamp column is required by the feature store, so one is added with a current timestamp
    signups_preprocessed['EventTime'] = timestamp
    
    outcomes_preprocessed = wr.s3.read_csv(
        path=f's3://{afd_bucket}/{afd_prefix}/data/preprocessed/outcomes', 
        dataset=True, 
#         index_col=0,
        dtype=outcomes_dtypes)
    
    outcomes_preprocessed['EventTime'] = timestamp
    
    print('Complete')

We now have a set of Pandas DataFrames that contain the signup and fraud outcome data. 

In [ ]:
signups_preprocessed.isnull().sum()

In [ ]:
signups_preprocessed.head()

In [ ]:
outcomes_preprocessed.isnull().sum()

In [ ]:
outcomes_preprocessed.head()

## 4. SageMaker Feature Store <a id='aud-feature-store'></a>

[overview](#all-up-overview)
___
Amazon SageMaker Feature Store is a purpose-built repository where you can store and access features so it’s much easier to name, organize, and reuse them across teams. SageMaker Feature Store provides a unified store for features during training and real-time inference without the need to write additional code or create manual processes to keep features consistent. SageMaker Feature Store keeps track of the metadata of stored features (e.g. feature name or version number) so that you can query the features for the right attributes in batches or in real time using Amazon Athena, an interactive query service. SageMaker Feature Store also keeps features updated, because as new data is generated during inference, the single repository is updated so new features are always available for models to use during training and inference.

A feature store consists of an offline componet stored in S3 and an online component stored in a low-latency database. The online database is optional, but very useful if you need supplemental features to be available at inference. In this section, we will create a feature groups for our Claims and Customers datasets. After inserting the claims and customer data into their respective feature groups, you need to query the offline store with Athena to build the training dataset.

You can reference the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) for more information about the SageMaker Feature Store.


In [ ]:
featurestore_runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime', 
    region_name=region
)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

### 4.1 Configure the feature groups
Feature data types can also be set via a config variable `feature_definitions`, but it will have to match the correspongin Python data type in the Pandas dataframe when it's ingested to the Feature Group. We will define the schemas for both our data sets in order to use it in the Feature Group

The following is a list of the feature names and data types of the final dataset that will be produced when your data flow is used to process your input dataset.

In [ ]:
signups_schema = [
    {
        "name": "ip_address",
        "type": "string"
    },
    {
        "name": "email_address",
        "type": "string"
    },
    {
        "name": "user_agent",
        "type": "string"
    },
    {
        "name": "customer_city",
        "type": "string"
    },
    {
        "name": "customer_state",
        "type": "string"
    },
    {
        "name": "customer_postal",
        "type": "float"
    },
    {
        "name": "customer_name",
        "type": "string"
    },
    {
        "name": "customer_address",
        "type": "string"
    },
    {
        "name": "phone_number",
        "type": "string"
    },
    {
        "name": "EVENT_TIMESTAMP",
        "type": "string"
    },
    {
        "name": "EventTime",
        "type": "float"
    }
]

outcomes_schema = [
    {
        "name": "ip_address",
        "type": "string"
    },
    {
        "name": "email_address",
        "type": "string"
    },
    {
        "name": "EVENT_LABEL",
        "type": "string"
    },
    {
        "name": "EventTime",
        "type": "float"
    }
]

Below we create the SDK input for those feature definitions. Some schema types in Data Wrangler are not 
supported by Feature Store. The following will create a default_FG_type set to String for these types.

In [ ]:
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

# the signups data set schema
signups_feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in signups_schema
]

# the outcome dataset schema
outcomes_feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in outcomes_schema
]

In [ ]:
signups_fg_name = f'{afd_prefix}-signups'
outcomes_fg_name = f'{afd_prefix}-outcomes'
%store signups_fg_name 
%store outcomes_fg_name

signups_feature_group = FeatureGroup(
    name=signups_fg_name, 
    sagemaker_session=feature_store_session,
    feature_definitions=signups_feature_definitions)

outcomes_feature_group = FeatureGroup(
    name=outcomes_fg_name, 
    sagemaker_session=feature_store_session,
    feature_definitions=outcomes_feature_definitions)


### 4.2 Create the feature groups
You must tell the Feature Group which columns in the dataframe correspond to the required record indentifier and event time features.

In [ ]:
print(f"{signups_fg_name} -- {outcomes_fg_name} are the feature group names in use")

In [ ]:
record_identifier_feature_name = 'ip_address' #we will use 'ip_address' as the record identifier for both datasets
event_time_feature_name = 'EventTime'

try:
    print(f"\n Using s3://{afd_bucket}/{afd_prefix}")
    signups_feature_group.create(
        s3_uri=f"s3://{afd_bucket}/{afd_prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=sagemaker_role,
        enable_online_store=True
    )
    print(f'Create "signups" feature group: SUCCESS')
except Exception as e:
    code = e.response.get('Error').get('Code')
    if code == 'ResourceInUse':
        print(f'Using existing feature group: {signups_fg_name}')
    else:
        raise(e)

try:
    outcomes_feature_group.create(
        s3_uri=f"s3://{afd_bucket}/{afd_prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=sagemaker_role,
        enable_online_store=True
    )
    print(f'Create "outcomes" feature group: SUCCESS')
except Exception as e:
    code = e.response.get('Error').get('Code')
    if code == 'ResourceInUse':
        print(f'Using existing feature group: {outcomes_fg_name}')
    else:
        raise(e)

Wait until feature group creation has fully completed

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")
    
wait_for_feature_group_creation_complete(feature_group=signups_feature_group)
wait_for_feature_group_creation_complete(feature_group=outcomes_feature_group)

### 4.3 Ingest records into the Feature Groups
After the Feature Groups have been created, we can put data into each store by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to s3 in chunks. The files will be written to the offline store within a few minutes of ingestion.

In [ ]:
if 'training_data_table' in locals():
    print("You may have already ingested the data into your Feature Groups. If you'd like to do this again, you can run the ingest methods outside of the 'if/else' statement.")

else:
    signups_feature_group.ingest(
    data_frame=signups_preprocessed, max_workers=3, wait=True
    );

    outcomes_feature_group.ingest(
        data_frame=outcomes_preprocessed, max_workers=3, wait=True
    );

Wait for offline store data to become available
This usually takes 5-8 minutes. The Feature Store offline database works with S3 Athena query engine and generates parquet files which can then be queried using SQL like Athena query.

In [ ]:
signups_feature_group_s3_prefix = f'{afd_prefix}/{account_id}/sagemaker/{region}/offline-store/{signups_fg_name}/data'
outcomes_feature_group_s3_prefix = f'{afd_prefix}/{account_id}/sagemaker/{region}/offline-store/{outcomes_fg_name}/data'

offline_store_contents = None
while (offline_store_contents is None):
    objects_in_bucket = s3_client.list_objects(Bucket=afd_bucket, Prefix=signups_feature_group_s3_prefix)
    if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) > 1):
        offline_store_contents = objects_in_bucket['Contents']
    else:
        print('Waiting for data in offline store...')
        time.sleep(60)
    
print('\nData available.')

Our feature store has now been created and data has been ingested. Now we will use our SageMaker Feature Store offline data store to create training dataset.

<img src="images/feature_store.png" width="800"/>

## 5. Create training dataset <a id='dataset'></a>

[overview](#all-up-overview)
___
Once the data is available in the offline store, it will automatically be cataloged and loaded into an Athena table (this is done by default, but can be turned off). In order to build our training and test datasets, you will submit a SQL query to join the the Signup attempts and outcomes tables created in Athena.

In [ ]:
signup_attempts_query = signups_feature_group.athena_query()
signup_outcomes_query = outcomes_feature_group.athena_query()

signup_attempts_table = signup_attempts_query.table_name
signup_outcomes_table = signup_outcomes_query.table_name
afd_database_name = signup_outcomes_query.database

%store signup_attempts_table
%store signup_outcomes_table
%store afd_database_name

feature_columns = list( set(signups_preprocessed.columns) ^ set(outcomes_preprocessed.columns) )
feature_columns_string = ", ".join(f'\"{c}\"' for c in feature_columns)
feature_columns_string = f'"EVENT_TIMESTAMP", "EVENT_LABEL", "{signup_attempts_table}".ip_address as ip_address, "{signup_attempts_table}".email_address as email_address, "user_agent" ,"customer_name", "phone_number", "customer_city", "customer_postal", "customer_state", "customer_address"' #+ feature_columns_string

query_string = f"""
SELECT DISTINCT {feature_columns_string}
FROM "{signup_attempts_table}" LEFT JOIN "{signup_outcomes_table}" 
ON "{signup_attempts_table}".ip_address = "{signup_outcomes_table}".ip_address
AND "{signup_attempts_table}".email_address = "{signup_outcomes_table}".email_address
"""


In [ ]:
response=signup_attempts_query.run(query_string=query_string, output_location=f's3://{afd_bucket}/{afd_prefix}/query_results')
signup_attempts_query.wait()
dataset = signup_attempts_query.as_dataframe()

output_file_name=f"{response}.csv"
output_file_path=f's3://{afd_bucket}/{afd_prefix}/query_results/{output_file_name}'

In [ ]:
print(output_file_path)

In [ ]:
dataset.head()

The query results will now be saved to the S3 bucket under `query_results/` prefix. The final training data file `afd_training_data.csv` is also included in this project. This concludes our data preperation for Amazon Fraud Detector. In the next notebook, we will build an AFD Model and train it with this dataset. The subsequent code cells will take the query output and move it to the final training data S3 location.

In [ ]:
import s3fs   # This is required to read CSV data directly from S3 into Pandas dataframe
import pandas as pd
import csv

df_train = pd.read_csv(output_file_path)
df_train.to_csv("./data/afd_training_data.csv", index=False)

In [ ]:
s3_client.upload_file(Filename='data/afd_training_data.csv', 
                      Bucket=afd_bucket, 
                      Key=f'{afd_prefix}/training_data/afd_training_data.csv')

## 5. Conclusion <a id='conclusion'></a>

[overview](#all-up-overview)
___

In this notebook, we processed our raw data with SageMaker Data Wrangler and then stored our features into SageMaker Feature store. We then ingested the preprocessed data into Feature Store offline datastore. We then used Athena query to join our pre-processed datasets to generate our training dataset and copied it into an S3 location. 

In the next notebook, we will be setting up our Amazon Fraud Detector model.